In [1]:
import pandas as pd
import numpy as np

In [2]:
path_train_data = '../../0raw_data/2010-2011 Solar home electricity data.csv'
path_eval_data = '../../0raw_data/2011-2012 Solar home electricity data.csv'
path_test_data = '../../0raw_data/2012-2013 Solar home electricity data.csv'

In [3]:
def loadTrainEvalTest(pathTrain,pathEval,pathTest):
    df_load_train = pd.read_csv(pathTrain, header=1, parse_dates=['date'], date_format="%d/%m/%y")
    df_load_eval = pd.read_csv(pathEval, header=1, parse_dates=['date'], date_format="%d/%m/%y")
    df_load_test = pd.read_csv(pathTest, header=1, parse_dates=['date'], date_format="%d/%m/%y")

    return pd.concat([df_load_train, df_load_eval, df_load_test])

In [4]:
def combineControlledAndGeneralConsumption(df):
    if (df == 'CL').any().any():
        combinedLoad = pd.DataFrame(columns=df.columns)
        cl = df[df['Consumption Category'] == 'CL'].set_index(pd.RangeIndex(0, 1096))
        gc = df[df['Consumption Category'] == 'GC'].set_index(pd.RangeIndex(0, 1096))
        clgc = pd.concat([cl,gc])
        for i in range(0,1096):
            sum = clgc.loc[i].sum()
            sum['date'] = i
            combinedLoad.loc[i] = sum
        final_load = combinedLoad
    else:
        final_load = df[df['Consumption Category']=='GC']
    
    load_array = np.array(final_load.drop(columns=['Consumption Category', 'date'])).flatten()
    return pd.DataFrame({"Load":load_array}) 

In [7]:
# Load data
df = loadTrainEvalTest(path_train_data,path_eval_data,path_test_data)

# Remove unneccessary data
df = df.drop(columns=['Generator Capacity', 'Postcode', 'Row Quality'])

# Choose customer
customer = 1
customer_df = df[df['Customer']==customer].drop(columns=['Customer'])

# Combine controlled and general consumption if needed
proccessed_df = combineControlledAndGeneralConsumption(customer_df)

# Get pv prodcution
pv_array = np.array(customer_df[customer_df['Consumption Category']=='GG'].drop(columns=['Consumption Category', 'date']).set_index(pd.RangeIndex(0, 1096))).flatten()
proccessed_df.insert(1,"PV",pv_array)

# Save df
proccessed_df.to_csv("../processed_load_pv_"+str(customer)+".csv", sep=',', index=False, encoding='utf-8')